# Subsetting MMETSP samples by transcriptome evaluation metrics

# Poor

* Problems running Trinity
* n contigs (large differences between NCGR and DIB)
* unique k-mers (k=25)
* annotations
* Complete BUSCO
* % ORF

In [10]:
%matplotlib inline
%pylab inline
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
import palettable as pal
import matplotlib as mpl
import numpy

Populating the interactive namespace from numpy and matplotlib


In [11]:
special_flowers = ['MMETSP0693', 'MMETSP1019', 'MMETSP0923', 'MMETSP0008', 'MMETSP1002', 'MMETSP1325', 'MMETSP1018', 
                   'MMETSP1346', 'MMETSP0088', 'MMETSP0092', 'MMETSP0717', 'MMETSP0223', 'MMETSP0115', 'MMETSP0196', 
                   'MMETSP0197', 'MMETSP0398', 'MMETSP0399', 'MMETSP0922']

In [3]:
unique_names = pd.read_csv("../assembly_evaluation_data/unique_gene_names_ncgr_dib.csv")

In [125]:
giant_matrix = "../assembly_evaluation_data/MMETSP_all_evaluation_matrix_2.csv"
all_metrics = pd.read_csv(giant_matrix)
#all_metrics.columns.values
# transrate score metrics
all_metrics = all_metrics[~all_metrics['SampleName'].isin(special_flowers)]
ncgr_cds_transrate_score = all_metrics[['SampleName', 'score.y','mean_orf_percent.y']]
ncgr_nt_transrate_score = all_metrics[['SampleName','score.x']]
dib_transrate_score = all_metrics[['SampleName','score']]
ncgr_nt_transrate_score.columns = ['SampleName','score']
ncgr_cds_transrate_score.columns = ['SampleName','score','mean_orf_percent']
#transrate reference metrics
ncgr_transrate_nt = all_metrics[['SampleName','p_refs_with_CRBB.y','n_seqs.x','mean_orf_percent.x']]
dib_transrate_nt = all_metrics[['SampleName','p_refs_with_CRBB','n_seqs','mean_orf_percent']]
ncgr_transrate_nt.columns = ['SampleName','p_refs_with_CRBB','n_seqs','mean_orf_percent']
# BUSCO
dib_busco_v3_euk = all_metrics[['SampleName','DIB.Complete_BUSCO_perc_euk_v3']]
dib_busco_v3_protist = all_metrics[['SampleName','DIB.Complete_BUSCO_perc_protist_v3']]
ncgr_busco_v3_euk = all_metrics[['SampleName','NCGR.Complete_BUSCO_perc_euk_v3']]
ncgr_busco_v3_protist = all_metrics[['SampleName','NCGR.Complete_BUSCO_perc_protist_v3']]
# unique dammit names
unique_dammit_names = pd.read_csv("../assembly_evaluation_data/normalized_unique_gene_names_ncgr_dib.csv")
unique_dammit_names = unique_dammit_names[~unique_dammit_names['MMETSP_id'].isin(special_flowers)]
# kmers
ncgr_nt_kmers = pd.read_table("../assembly_evaluation_data/unique_kmers_ncgr_nt.txt",header=0,sep='\t')
ncgr_nt_kmers.columns = ['SampleName','unique_kmers_ncgr_nt']
dib_ncgr_kmers = all_metrics[["SampleName","Phylum","dib_assembly_unique_kmers","ncgr_assembly_unique_kmers"]]
dib_ncgr_kmers.columns = ["SampleName","Phylum","unique_kmers_dib","unique_kmers_ncgr_cds"]
dib_ncgr_kmers = pd.merge(ncgr_nt_kmers,dib_ncgr_kmers,on='SampleName')

In [14]:
#ncgr_cds_transrate_score = ncgr_cds_transrate_score.drop_duplicates()
#ncgr_nt_transrate_score = ncgr_nt_transrate_score.drop_duplicates()
#dib_transrate_score = dib_transrate_score.drop_duplicates()
ncgr_cds_transrate_score = ncgr_cds_transrate_score.dropna()
ncgr_nt_transrate_score = ncgr_nt_transrate_score.dropna()
dib_transrate_score = dib_transrate_score.dropna()

In [22]:
# dammit gene names comparison
gene_names = unique_names.drop_duplicates()
# giant matrix of all metrics, including phyla
#all_metrics = all_metrics.drop_duplicates()

# n contigs

In [37]:
df1==ncgr_transrate_nt
df2=dib_transrate_nt
combined_df = pd.merge(df1, df2, left_index=True, right_index=True)
low_dib_contigs = combined_df.loc[combined_df['n_seqs_x'] > combined_df['n_seqs_y']]['SampleName_x']
print(low_dib_contigs.tolist())

['MMETSP0013', 'MMETSP0045', 'MMETSP0121', 'MMETSP0132', 'MMETSP0229', 'MMETSP0268', 'MMETSP0329', 'MMETSP0378', 'MMETSP0380', 'MMETSP0759', 'MMETSP0932', 'MMETSP1148']


# % ORF

In [156]:
df1=ncgr_transrate_nt
df2=dib_transrate_nt
combined_df = pd.merge(df1, df2, left_index=True, right_index=True)
low_dib_ORF = combined_df.loc[combined_df['mean_orf_percent_x'] > combined_df['mean_orf_percent_y']]['SampleName_x']
print(low_dib_ORF.tolist())

['MMETSP0010', 'MMETSP0011', 'MMETSP0013', 'MMETSP0014', 'MMETSP0015', 'MMETSP0017', 'MMETSP0034', 'MMETSP0038', 'MMETSP0040', 'MMETSP0044', 'MMETSP0046', 'MMETSP0098', 'MMETSP0132', 'MMETSP0229', 'MMETSP0347', 'MMETSP0359', 'MMETSP0361', 'MMETSP0451', 'MMETSP0469', 'MMETSP0780', 'MMETSP0807', 'MMETSP0886', 'MMETSP0887', 'MMETSP0888', 'MMETSP0915', 'MMETSP0916', 'MMETSP0917', 'MMETSP0924', 'MMETSP1012', 'MMETSP1055', 'MMETSP1103', 'MMETSP1147', 'MMETSP1148', 'MMETSP1163', 'MMETSP1166', 'MMETSP1329', 'MMETSP1344', 'MMETSP1359', 'MMETSP1397', 'MMETSP1426', 'MMETSP1433', 'MMETSP1435']


# BUSCO

In [106]:
df1=ncgr_busco_v3_euk.dropna()
df2=dib_busco_v3_euk.dropna()
combined_df = pd.merge(df1, df2,on='SampleName')
column='Complete_BUSCO_perc'
low_dib_BUSCO = combined_df.loc[combined_df['NCGR.Complete_BUSCO_perc_euk_v3'] > combined_df['DIB.Complete_BUSCO_perc_euk_v3']]['SampleName']
#print(low_dib_contigs.tolist())
print(len(low_dib_BUSCO.tolist()))

381


In [107]:
# Where difference is more than 30% between NCGR and DIB
df1=ncgr_busco_v3_euk
df2=dib_busco_v3_euk
df2=dib_busco_v3_euk
column='Complete_BUSCO_perc'
df1name = 'df1'
df2name = 'df2'
df1.columns = ['SampleName', 'Complete_BUSCO_perc_NCGR']
df2.columns = ['SampleName', 'Complete_BUSCO_perc_DIB']
combined_df = pd.merge(df1, df2,on='SampleName')
print(combined_df.shape)
print('NCGR:',len(combined_df.loc[combined_df['Complete_BUSCO_perc_NCGR'] > combined_df['Complete_BUSCO_perc_DIB']]))
print('DIB:',len(combined_df.loc[combined_df['Complete_BUSCO_perc_NCGR'] < combined_df['Complete_BUSCO_perc_DIB']]))
print('DIB = NCGR:',len(combined_df.loc[combined_df['Complete_BUSCO_perc_NCGR'] == combined_df['Complete_BUSCO_perc_DIB']]))
diff = combined_df.loc[(combined_df['Complete_BUSCO_perc_NCGR'] - combined_df['Complete_BUSCO_perc_DIB']) > 30]
low_BUSCO = diff['SampleName'].tolist()
print(low_BUSCO)

(660, 3)
NCGR: 381
DIB: 235
DIB = NCGR: 42
['MMETSP0045', 'MMETSP0121', 'MMETSP0169', 'MMETSP0232', 'MMETSP0329', 'MMETSP0439', 'MMETSP0932']


# annotations

# Low unique dammit names in DIB

In [96]:
unique_dammit_names.head()

,Unnamed: 0,MMETSP_id,NCGR,DIB
0,0,MMETSP1385,0.207904,0.355840
1,1,MMETSP1144,0.179560,0.291139
2,2,MMETSP0471,0.107027,0.270018
3,3,MMETSP0957,0.134107,0.376607
4,4,MMETSP1058,0.076528,0.174923


In [97]:
unique_dammit_names_dib_low = unique_dammit_names.loc[unique_dammit_names['DIB'] - unique_dammit_names['NCGR'] < 0]
print(unique_dammit_names_dib_low.shape)
print(unique_dammit_names_dib_low)

(9, 4)
     Unnamed: 0   MMETSP_id      NCGR       DIB
17           17  MMETSP1412  0.996745  0.500000
99           99  MMETSP0380  0.327761  0.274651
101         101  MMETSP0378  0.271208  0.265763
141         141  MMETSP0232  0.974898  0.661442
190         190  MMETSP1148  0.727273  0.558824
240         240  MMETSP0912  0.860795  0.241117
258         258  MMETSP0169  0.450119  0.409318
261         261  MMETSP0937  0.896324  0.212418
272         272  MMETSP1057  0.959008  0.352941


In [98]:
print(unique_dammit_names_dib_low['MMETSP_id'].tolist())

['MMETSP1412', 'MMETSP0380', 'MMETSP0378', 'MMETSP0232', 'MMETSP1148', 'MMETSP0912', 'MMETSP0169', 'MMETSP0937', 'MMETSP1057']


# unique k-mers

In [129]:
dib_ncgr_kmers.head()

,SampleName,unique_kmers_ncgr_nt,Phylum,unique_kmers_dib,unique_kmers_ncgr_cds
0,MMETSP1403,14078915,Chlorophyta,14970841,12339926.0
1,MMETSP0292,10829338,Ochrophyta,22905647,8254399.0
2,MMETSP1316,10049557,Chlorophyta,11892637,9050349.0
3,MMETSP1042,20957422,Cryptophyta,28583565,19925214.0
4,MMETSP0228,49090117,Dinophyta,70670535,45605742.0


In [131]:
dib_low_kmers = dib_ncgr_kmers.loc[dib_ncgr_kmers['unique_kmers_dib'] - dib_ncgr_kmers['unique_kmers_ncgr_nt'] < 0]
low_kmers = dib_low_kmers['SampleName'].tolist()

In [158]:
low_contigs = low_dib_contigs.tolist()
low_unique_annotation_names = unique_dammit_names_dib_low['MMETSP_id'].tolist()
#low_BUSCO
low_ORF = low_dib_ORF.tolist()
#low_kmers

In [257]:
low_MMETSP = low_kmers + low_unique_annotation_names + low_BUSCO + low_ORF + low_kmers

In [263]:
print(sorted(set(low_MMETSP)))

['MMETSP0010', 'MMETSP0011', 'MMETSP0013', 'MMETSP0014', 'MMETSP0015', 'MMETSP0017', 'MMETSP0034', 'MMETSP0038', 'MMETSP0040', 'MMETSP0044', 'MMETSP0045', 'MMETSP0046', 'MMETSP0098', 'MMETSP0113', 'MMETSP0121', 'MMETSP0132', 'MMETSP0152', 'MMETSP0169', 'MMETSP0229', 'MMETSP0232', 'MMETSP0252', 'MMETSP0268', 'MMETSP0315', 'MMETSP0329', 'MMETSP0347', 'MMETSP0359', 'MMETSP0361', 'MMETSP0372', 'MMETSP0378', 'MMETSP0380', 'MMETSP0439', 'MMETSP0451', 'MMETSP0469', 'MMETSP0780', 'MMETSP0807', 'MMETSP0886', 'MMETSP0887', 'MMETSP0888', 'MMETSP0891', 'MMETSP0912', 'MMETSP0915', 'MMETSP0916', 'MMETSP0917', 'MMETSP0924', 'MMETSP0930', 'MMETSP0932', 'MMETSP0937', 'MMETSP1012', 'MMETSP1055', 'MMETSP1057', 'MMETSP1065', 'MMETSP1103', 'MMETSP1147', 'MMETSP1148', 'MMETSP1163', 'MMETSP1166', 'MMETSP1178', 'MMETSP1329', 'MMETSP1344', 'MMETSP1358', 'MMETSP1359', 'MMETSP1397', 'MMETSP1412', 'MMETSP1426', 'MMETSP1433', 'MMETSP1435']


In [265]:
print(len(sorted(set(low_MMETSP))))

66


# Good

* Complete BUSCO > 80%
* % ORF > 90%
* length transcripts > 20k
* unique annotations > 50%
* unique k-mers (k=25) > 50 million more than NCGR

# BUSCO > 90%

In [387]:
# Greater than 80% Complete
df2=dib_busco_v3_euk
df2.columns = ['SampleName', 'Complete_BUSCO_perc_DIB']
high_DIB = df2.loc[df2['Complete_BUSCO_perc_DIB'] > 90]['SampleName']
#print(high_DIB.tolist())
print(len(high_DIB.tolist()))
high_DIB_BUSCO = high_DIB.tolist()

8


# Transrate score

In [390]:
df2=dib_transrate_score
high_dib_transrate = df2.loc[df2['score'] > 0.5]['SampleName']
#print(low_dib_contigs.tolist())
print(len(high_dib_transrate.tolist()))
high_dib_transrate = high_dib_transrate.tolist()

3


In [391]:
BUSCO_transrate = high_dib_transrate + high_DIB_BUSCO
BUSCO_transrate = list(set(BUSCO_transrate))
len(BUSCO_transrate)

11

# % ORF > 90%

In [346]:
df2=dib_transrate_nt
high_dib_ORF = df2.loc[df2['mean_orf_percent'] > 90]['SampleName']
#print(high_dib_ORF.tolist())
high_dib_ORF = high_dib_ORF.tolist()
print(len(high_dib_ORF))

131


In [359]:
BUSCO_ORF = high_dib_ORF + high_DIB_BUSCO
BUSCO_ORF = list(set(BUSCO_ORF))
len(BUSCO_ORF)

139

# Length > 30K bases

In [347]:
DIB_longest = all_metrics[['SampleName','largest.x']]
DIB_longest = DIB_longest.loc[DIB_longest['largest.x'] > 30000]['SampleName']
print(len(DIB_longest.tolist()))
DIB_longest = DIB_longest.tolist()

6


# Unique normalized annotated contigs > 70%

In [348]:
high_unique_annotation_names = unique_dammit_names.loc[unique_dammit_names['DIB'] > 0.7]['MMETSP_id']
print(len(high_unique_annotation_names.tolist()))
high_unique_annotation_names = high_unique_annotation_names.tolist()

5


# Unique k-mers, more than 30 million more unique k-mers in DIB vs. NCGR

In [349]:
dib_high_kmers = dib_ncgr_kmers.loc[dib_ncgr_kmers['unique_kmers_dib'] - dib_ncgr_kmers['unique_kmers_ncgr_nt'] > 30000000]
dib_high_kmers = dib_high_kmers['SampleName'].tolist()
print(len(dib_high_kmers))

34


In [369]:
high_MMETSP = high_DIB_BUSCO

In [370]:
print(len(sorted(list(set(high_MMETSP)))))

8


In [371]:
len(list(set(low_MMETSP + high_MMETSP)))

70